In [3]:
!pip install dagshub mlflow

In [4]:
import dagshub
dagshub.init(repo_owner='lkhok22', repo_name='ML-hw2', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c5ea38e7-8958-4321-a945-a502c5c71166&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=612944d76dfe0ca97bdc25b0b1877873769d1077ae61e6f9c891f6ba71b7f279




Output()

Accessing as lkhok22

Initialized MLflow to track repo "lkhok22/ML-hw2"

Repository lkhok22/ML-hw2 initialized!

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [6]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

# Read and merge, Reduce memory

In [7]:
df_train_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
df_train_tr = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
df_test_id = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
df_test_tr = pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')

In [8]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [9]:
df_train = pd.merge(df_train_tr, df_train_id, on='TransactionID', how='left')
df_test = pd.merge(df_test_tr, df_test_id, on='TransactionID', how='left')

In [10]:
del df_train_id, df_train_tr
del df_test_id,df_test_tr

In [11]:
train = reduce_mem_usage(df_train)
test = reduce_mem_usage(df_test)

Memory usage of dataframe is 1955.37 MB
Memory usage after optimization is: 645.97 MB
Decreased by 67.0%
Memory usage of dataframe is 1673.87 MB
Memory usage after optimization is: 561.50 MB
Decreased by 66.5%


In [12]:
D_features = ['D4','D6','D11','D12','D14','D15']
sum = 0
for i in D_features:
  filt = train[train[i]<0.0].index
  
  for j in filt:
    train=train.drop(index=j)


train.drop(["M1"], axis=1, inplace=True)
test.drop(["M1"], axis=1, inplace=True)

In [13]:
X=df_train.drop(columns=['isFraud'])
y=df_train['isFraud']


In [14]:
from sklearn.model_selection import train_test_split

#80% train, 20% test

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [15]:
train_ids = X_train.pop('TransactionID')
test_ids = X_test.pop('TransactionID')

# Cleaning

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
class DropNullColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.7):
        self.threshold = threshold
        self.columns_to_drop_ = []

    def fit(self, X, y=None):
        null_ratios = X.isna().mean()
        self.columns_to_drop_ = null_ratios[null_ratios > self.threshold].index.tolist()
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop_, errors='ignore')


In [17]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class FillMissingValues(BaseEstimator, TransformerMixin):
    def __init__(self, strategy_num='mean', strategy_cat='mode'):
        self.strategy_num = strategy_num
        self.strategy_cat = strategy_cat
        self.num_fill_values_ = {}
        self.cat_fill_values_ = {}

    def fit(self, X, y=None):
        num_cols = X.select_dtypes(include=['float64', 'int64']).columns
        cat_cols = X.select_dtypes(include=['object', 'category']).columns

        for col in num_cols:
            if self.strategy_num == 'mean':
                self.num_fill_values_[col] = X[col].mean()
            elif self.strategy_num == 'median':
                self.num_fill_values_[col] = X[col].median()

        for col in cat_cols:
            if self.strategy_cat == 'mode':
                self.cat_fill_values_[col] = X[col].mode()[0]

        return self

    def transform(self, X):
        X_filled = X.copy()
        for col, fill_value in self.num_fill_values_.items():
            X_filled[col] = X_filled[col].fillna(fill_value)
        for col, fill_value in self.cat_fill_values_.items():
            X_filled[col] = X_filled[col].fillna(fill_value)
        return X_filled


# Feature Engineering

In [18]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

class LabelEncodeCategoricals(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}

    def fit(self, X, y=None):
        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        for col in cat_cols:
            le = LabelEncoder()
            X_col = X[col].astype(str).fillna('NA')
            le.fit(X_col)
            self.encoders[col] = le
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for col, le in self.encoders.items():
            X_encoded[col] = le.transform(X_encoded[col].astype(str).fillna('NA'))
        return X_encoded


# Feature Selection

In [19]:
class FeatureSelectorByImportance(BaseEstimator, TransformerMixin):
    def __init__(self, model, percentile=30):
        self.model = model
        self.percentile = percentile

    def fit(self, X, y):
        self.model.fit(X, y)
        importances = self.model.feature_importances_
        threshold = np.percentile(importances, 100 - self.percentile)

        # If X is a DataFrame, use column names, otherwise make generic names
        if hasattr(X, 'columns'):
            self.feature_names_ = X.columns
        else:
            self.feature_names_ = [f"feature_{i}" for i in range(X.shape[1])]

        self.selected_features_ = np.array(self.feature_names_)[importances >= threshold]
        return self

    def transform(self, X):
        # Same: if X is DataFrame, use columns, else use indices
        if hasattr(X, 'loc'):
            return X.loc[:, self.selected_features_]
        else:
            selected_indices = [i for i, name in enumerate(self.feature_names_) if name in self.selected_features_]
            return X[:, selected_indices]


# Training and Evaluation

In [20]:
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

# Set experiment
mlflow.set_experiment("GradientBoosting")

# Preprocessor: handles NaNs FIRST
preprocessor = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='median'), make_column_selector(dtype_include=['int64', 'float64'])),
    ('cat', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ]), make_column_selector(dtype_include=['object', 'category']))
])

# Full pipeline
pipeline = Pipeline([
    ('drop_nulls', DropNullColumns(threshold=0.9)),  # drop columns with too many nulls
    ('preprocessing', preprocessor),                # THEN impute + encode
    ('feature_selection', FeatureSelectorByImportance( # THEN select features
        model=DecisionTreeClassifier(max_depth=5, random_state=42),
        percentile=30
    )),
    ('model', GradientBoostingClassifier(             
        n_estimators=100,
        learning_rate=0.5,
        random_state=42
    ))
])

# MLflow logging
with mlflow.start_run(run_name="GradientBoosting1"):

    # Fit
    pipeline.fit(X_train, y_train)

    # Predict
    train_preds = pipeline.predict_proba(X_train)[:, 1]
    test_preds = pipeline.predict_proba(X_test)[:, 1]

    # Score
    train_auc = roc_auc_score(y_train, train_preds)
    test_auc = roc_auc_score(y_test, test_preds)

    # Log params
    mlflow.log_param("model", "GradientBoosting")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.5)
    mlflow.log_param("random_state", 42)

    # Log metrics
    mlflow.log_metric("train_auc", train_auc)
    mlflow.log_metric("test_auc", test_auc)

    # Log model
    mlflow.sklearn.log_model(pipeline, "gradientboosting_full_pipeline")

    print(f"Train AUC: {train_auc:.4f}")
    print(f"Test AUC: {test_auc:.4f}")


2025/04/27 19:35:18 INFO mlflow.tracking.fluent: Experiment with name 'GradientBoosting' does not exist. Creating a new experiment.
2025/04/27 19:36:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Train AUC: 0.7683
Test AUC: 0.7758
🏃 View run GradientBoosting1 at: https://dagshub.com/lkhok22/ML-hw2.mlflow/#/experiments/9/runs/00e58f40f1da43b9a82e84d755ee4e7a
🧪 View experiment at: https://dagshub.com/lkhok22/ML-hw2.mlflow/#/experiments/9
